In [4]:
import cv2
from keras.models import load_model
import numpy as np
from datetime import datetime, timedelta

USER_OPTIONS_EXPIRY_TIME = 5

ROCK = 0
PAPER = 1
SCISSOR = 2

def getExpiryTime(second=USER_OPTIONS_EXPIRY_TIME):
    return datetime.now() + timedelta(seconds=second)

def getWinningText(user_option, computer_option):
    if user_option == computer_option:
        return "Tie"
    elif user_option > computer_option:
        if computer_option == ROCK and user_option == SCISSOR:
            return "Computer wins"
        else:
            return "User wins"
    else:
        if user_option == ROCK and computer_option == SCISSOR:
            return "User wins"
        else:
            return "Computer wins"

def startGame():
    model = load_model('./model/keras_model.h5')
    cap = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX

    with open('./model/labels.txt') as f:
        lines = f.readlines()

    labels = []
    for line in lines:
        labels.append(line.split(" ")[1].replace("\n", ""))

    computer_class = np.random.randint(0, 3)
    computer_label = labels[computer_class]

    prev_choice = 3 # initial choice = None
    expiry_time = None
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    is_playing = True
    winning_text = ""

    while True:
        ret, frame = cap.read()

        if is_playing == False:
            cv2.putText(frame,
                        winning_text, 
                        (20, 50), 
                        font, 1, 
                        (255, 255, 255), 
                        2, 
                        cv2.LINE_4)

            cv2.putText(frame,
                    f'Press r to restart, press q to quit the game', 
                    (20, 100), 
                    font, 1, 
                    (255, 255, 255), 
                    2, 
                    cv2.LINE_4)

            cv2.imshow('frame', frame)
            key = cv2.waitKey(1) & 0xFF
            
            # Press q to close the window
            if key == ord("q"):
                break

            # Press r to restart the game
            if key == ord("r"):
                is_playing = True
                computer_class = np.random.randint(0, 3)
                computer_label = labels[computer_class]

                prev_choice = 3 # initial choice = None
                expiry_time = None
                
        else:
            cv2.putText(frame,
                f'Hold your options for {USER_OPTIONS_EXPIRY_TIME} seconds', 
                (20, 50), 
                font, 1, 
                (255, 255, 255), 
                2, 
                cv2.LINE_4)

            resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
            image_np = np.array(resized_frame)
            normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
            data[0] = normalized_image
            prediction = model.predict(data)
            cv2.imshow('frame', frame)
            # print(prediction)

            predict_class = np.argmax(np.squeeze(prediction))
            predict_label = labels[predict_class]

            print(predict_label)

            if predict_class < 3:
                if prev_choice != predict_class:
                    prev_choice = predict_class
                    expiry_time = getExpiryTime()

                    print(f"User select new option {predict_label}, update expiry time {expiry_time}")

                elif datetime.now() > expiry_time:
                    winning_text = getWinningText(predict_class, computer_class)
                    winning_text = f'Computer option is {computer_label}, your option is {predict_label}, {winning_text}'

                    is_playing = False

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break


    # After the loop release the cap object
    cap.release()
    # Destroy all the windows
    cv2.destroyAllWindows()

startGame()

2022-02-01 21:40:01.516627: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-01 21:40:01.516797: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Paper
User select new option Paper, update expiry time 2022-02-01 21:40:06.785131
Rock
User select new option Rock, update expiry time 2022-02-01 21:40:06.854199
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Paper
User select new option Paper, update expiry time 2022-02-01 21:40:09.656268
Paper
Paper
Paper
Paper
Scissor
User select new option Scissor, update expiry time 2022-02-01 21:40:09.990227
Scissor
Paper
User select new option Paper, update expiry time 2022-02-01 21:40:10.123247
Rock
User select new option Rock, update expiry time 2022-02-01 21:40:10.191533
Rock
Paper
User select new option Paper, update expiry time 2022-02-01 21:40:10.323606
Rock
User select new option Rock, update expiry time 2022-02-01 21:40:10.392096
Paper
User select new option Paper, update expiry time 2022-02-01 21:40:10.460782
Paper
Paper
Paper
Pape